Ceneo Scraper

# Lista kroków "manualnych"

1 Wejdź na stronę produktu / z opiniami o produkcie
2 Dla każdej opinii na stronie
    1 skopiuj opinie
    2 wklej opinie do edytora tekstu
3 Przejdź do kolejnej strony z opiniami
4 Powtarzaj kroki 2 i 3 dopóki są strony z opiniami

# Lista kroków scrapera

1 Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2 

63717975
44279952

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autora|autor|.user-post__author-name|
|rekomendację|recommendation|span.user-post__recomendation > em|
|data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|useful|.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|.vote-no > span|
|treść opinii|content|div.user-post__text|
|listę wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|listę zalet|pros|div.review-feature__title--positive ~ div.review-feature__item|
|liczba gwiazdek|stars|span.user-post__score-count|

1. Import bibliotek

In [27]:
import requests
import json
import os

2. Podanie adresu pierwszej strony z opiniami na podstawie kodu produktu

In [28]:

from bs4 import BeautifulSoup
product_code= input("Podaj kod produktu z ceneo.pl: ")
url = f"https://www.ceneo.pl/{product_code}#tab-reviews"


3. Pobranie wszystkich opinii o produkcie z serwisu ceneo.pl

In [29]:
def get_data(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if attribute:
        if selector:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
    try:
        return ancestor.select_one(selector).text.strip()
    except AttributeError:
        return None


In [30]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")

    for opinion in opinions:
        single_opinion = {
            'opinion_id': opinion["data-entry-id"].strip(),
            'author': get_data(opinion, ".user-post__author-name"),
            'recommendation': get_data(opinion, "span.user-post__recomendation > em"),
            'stars': get_data(opinion, "span.user-post__score-count"),
            'content': get_data(opinion, "div.user-post__text"),
            'pros': get_data(opinion, "div.review-feature__title--positives ~ div.  review-feature__item", None, True),
            'cons': get_data(opinion, "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
            'post_date': get_data(opinion, "span.user-post__published > time:nth-child(1)", "datetime"),
            'purchase_date': get_data(opinion, "span.user-post__published > time:nth-child(2)", "datetime"),
            'useful': get_data(opinion, ".vote-yes > span"),
            'useless': get_data(opinion, ".vote-no > span"),
        }
        all_opinions.append(single_opinion)
    
    try:
        url = "https://ceneo.pl"+page.select_one('a.pagination__next')['href']
    except TypeError:
        url = None

https://www.ceneo.pl/63717975#tab-reviews
https://ceneo.pl/63717975/opinie-2
https://ceneo.pl/63717975/opinie-3
https://ceneo.pl/63717975/opinie-4
https://ceneo.pl/63717975/opinie-5
https://ceneo.pl/63717975/opinie-6
https://ceneo.pl/63717975/opinie-7
https://ceneo.pl/63717975/opinie-8
https://ceneo.pl/63717975/opinie-9
https://ceneo.pl/63717975/opinie-10
https://ceneo.pl/63717975/opinie-11
https://ceneo.pl/63717975/opinie-12
https://ceneo.pl/63717975/opinie-13
https://ceneo.pl/63717975/opinie-14
https://ceneo.pl/63717975/opinie-15


5. Zapis opinii do pliku JSON

In [32]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()

Definicje stałych i globalnych